In [28]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns',None)#displaying long list of columns
pd.set_option('display.max_rows', None)#displaying long list of rows
pd.set_option('display.width', 1000)#width of window


In [30]:
trainfile = r'/gdrive/My Drive/CIS508-FALL2020/PT3A/train.csv'
trainData = pd.read_csv(trainfile)  #creates a dataframe
testfile = r'/gdrive/My Drive/CIS508-FALL2020/PT3A/test.csv'
testData = pd.read_csv(testfile)  #creates a dataframe
print(trainData.shape)
print(testData.shape)

(260753, 299)
(173836, 298)


In [31]:
#Extract Target Column before doing missing value substitutions and one-hot encoding======
Target_Train_Cols = trainData["QuoteConversion_Flag"]#make copy of target column
trainData = trainData.drop(["QuoteConversion_Flag"], axis=1) #extracting training data without the target column

print(trainData.shape)

(260753, 298)


In [32]:
#DROP COLUMNS WITH LOTS OF MISSING VALUES===============================
#CAN ALSO DROP ROWS WITH LOTS OF MISSING VALUES
#Combine Train data and test data first so that the SAME COLUMNS are DROPPED in each
combined_Data = pd.concat([trainData, testData], keys=[0,1])

combined_Data.isnull().sum().sort_values(ascending=False)


PropertyField29        334630
PersonalField84        207020
PropertyField38          2066
PersonalField7            182
PropertyField36           180
PropertyField3            150
PropertyField4            115
PropertyField34           111
PropertyField32           111
PropertyField5              1
PropertyField30             1
PersonalField54             0
PersonalField55             0
GeographicField64           0
PersonalField53             0
PersonalField57             0
PersonalField58             0
PersonalField59             0
PersonalField60             0
PersonalField61             0
PersonalField62             0
PersonalField63             0
PersonalField64             0
PersonalField65             0
PersonalField66             0
PersonalField56             0
PersonalField50             0
PersonalField52             0
PersonalField51             0
PersonalField68             0
PersonalField49             0
PersonalField48             0
PersonalField47             0
PersonalFi

In [33]:
#Define threshold for dropping columns
percent=int(0.6*(combined_Data.shape[0]))
print(percent)
#Drop columns that have less than "thresh" number of non_Nans
td1=combined_Data.dropna(thresh=percent,axis=1)
print(td1.shape)

260753
(434589, 296)


In [34]:
#look at what other columns have missing values
td1.isnull().sum().sort_values(ascending=False)

PropertyField38        2066
PersonalField7          182
PropertyField36         180
PropertyField3          150
PropertyField4          115
PropertyField34         111
PropertyField32         111
PropertyField5            1
PropertyField30           1
PersonalField55           0
PersonalField53           0
PersonalField54           0
PersonalField56           0
PersonalField51           0
PersonalField57           0
PersonalField58           0
PersonalField59           0
PersonalField60           0
PersonalField61           0
PersonalField62           0
PersonalField63           0
PersonalField64           0
PersonalField65           0
PersonalField52           0
GeographicField64         0
PersonalField50           0
PersonalField49           0
PersonalField67           0
PersonalField48           0
PersonalField47           0
PersonalField46           0
PersonalField45           0
PersonalField44           0
PersonalField43           0
PersonalField42           0
PersonalField41     

In [35]:
#NOW IMPUTE MISSING VALUES FOR THE OTHER COLUMNS=========================
#IMPUTE (SUBSTITUTE) MEAN VALUES FOR NaN IN NUMERIC COLUMNS 
numeric=td1.select_dtypes(include=['int','float64']).columns
for num in numeric:
  td1[num]=td1[num].fillna(td1[num].mean())

#IMPUTE (SUBSTITUTE) MODE VALUES FOR NaN IN CATEGORICAL COLUMNS
train_cat_cols = td1.select_dtypes(exclude=['int','float64']).columns#selecting the categorical columns
for colss in train_cat_cols:
  if(td1.iloc[0][colss]=="N"):
        td1[colss]=td1[colss].fillna("N")
  else:
    td1[colss]=td1[colss].fillna(td1[colss].mode())
  
print(td1.head(20))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

      QuoteNumber Original_Quote_Date Field6  Field7  Field8  Field9 Field10  Field11 Field12  CoverageField1A  CoverageField1B  CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  CoverageField4A  CoverageField4B  CoverageField5A  CoverageField5B  CoverageField6A  CoverageField6B CoverageField8 CoverageField9  CoverageField11A  CoverageField11B  SalesField1A  SalesField1B  SalesField2A  SalesField2B  SalesField3  SalesField4  SalesField5  SalesField6 SalesField7  SalesField8  SalesField9  SalesField10  SalesField11  SalesField12  SalesField13  SalesField14  SalesField15  PersonalField1  PersonalField2  PersonalField4A  PersonalField4B  PersonalField5  PersonalField6 PersonalField7  PersonalField8  PersonalField9  PersonalField10A  PersonalField10B  PersonalField11  PersonalField12  PersonalField13  PersonalField14  PersonalField15 PersonalField16 PersonalField17 PersonalField18 PersonalField19  PersonalField22  PersonalField23  PersonalField24  PersonalField25  \
0 0  

In [36]:
#CHECK IF THERE ARE ANY REMAINING MISSING VALUES 
td1.isnull().sum().sort_values(ascending=False)

PropertyField34        111
PropertyField32        111
PropertyField5           1
GeographicField64        0
PersonalField52          0
PersonalField54          0
PersonalField55          0
PersonalField56          0
PersonalField57          0
PersonalField58          0
PersonalField59          0
PersonalField60          0
PersonalField61          0
PersonalField62          0
PersonalField63          0
PersonalField64          0
PersonalField65          0
PersonalField66          0
PersonalField67          0
PersonalField53          0
PersonalField50          0
PersonalField51          0
PersonalField42          0
PersonalField36          0
PersonalField37          0
PersonalField38          0
PersonalField39          0
PersonalField40          0
PersonalField41          0
PersonalField43          0
PersonalField69          0
PersonalField44          0
PersonalField45          0
PersonalField46          0
PersonalField47          0
PersonalField48          0
PersonalField49          0
P

In [37]:
#DROP COLUMNS THAT STILL HAVE NULL VALUES
print(td1.shape)
td1=td1.drop(columns=["PropertyField32", "PropertyField34"])
td1=td1.drop(columns=["PropertyField5"])
td1.isnull().sum()
print(td1.shape)

(434589, 296)
(434589, 293)


In [39]:
#DO ONE-HOT ENCODING ON CATEGORICAL VARIABLES==============================================
#The below function returns a list of categorical features which are not numeric. 
train_cat_cols = td1.select_dtypes(exclude=['float','int']).columns #selecting the categorical columns
print(train_cat_cols.shape)
print(train_cat_cols)

#If there are categorical columns which are encoded as numeric ones 
#then we need to explicitly enter the column names in a list and concatenate the two lists in python.
#ONE-HOT ENCODING-generate one-hot encoding on a common basis -THIS TAKES 30 MINS

combined_Data = pd.get_dummies(td1,train_cat_cols)
combined_Data.head(10)


(25,)
Index(['Original_Quote_Date', 'Field6', 'Field10', 'Field12', 'CoverageField8', 'CoverageField9', 'SalesField7', 'PersonalField7', 'PersonalField16', 'PersonalField17', 'PersonalField18', 'PersonalField19', 'PropertyField3', 'PropertyField4', 'PropertyField7', 'PropertyField14', 'PropertyField28', 'PropertyField30', 'PropertyField31', 'PropertyField33', 'PropertyField36', 'PropertyField37', 'PropertyField38', 'GeographicField63', 'GeographicField64'], dtype='object')


QuoteNumber  Field7  Field8  Field9  Field11  CoverageField1A  CoverageField1B  CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  CoverageField4A  CoverageField4B  CoverageField5A  CoverageField5B  CoverageField6A  CoverageField6B  CoverageField11A  CoverageField11B  SalesField1A  SalesField1B  SalesField2A  SalesField2B  SalesField3  SalesField4  SalesField5  SalesField6  SalesField8  SalesField9  SalesField10  SalesField11  SalesField12  SalesField13  SalesField14  SalesField15  PersonalField1  PersonalField2  PersonalField4A  PersonalField4B  PersonalField5  PersonalField6  PersonalField8  PersonalField9  PersonalField10A  PersonalField10B  PersonalField11  PersonalField12  PersonalField13  PersonalField14  PersonalField15  PersonalField22  PersonalField23  PersonalField24  PersonalField25  PersonalField26  PersonalField27  PersonalField28  PersonalField29  PersonalField30  PersonalField31  PersonalField32  PersonalField33  PersonalField34  PersonalField35  \
0 0            1      23  0.9403  0.0006   1.0200               17               23               17               23               15               22               16               22               13               22               13               23                 2                 1             7            18             3             8            0            5            5           24        48649            0             0             0             0             0             0             0               0               0               -1               -1               7               0               1               2                 5                 7                0                1                2                2               24                1                0                0                0                0                1                2                2                0                1                1                1                1                0   
  1            2       7  1.0006  0.0040   1.2433                6                8                6                8                5                7                5                8               13               22               13               23                 5                 9             5            14             6            18            1            5            5           11        26778            0             0             1             1             0             0             0               1               1               14               19               7               0               1               2                24                25                0                1                2                5               19                1                0                0                0                0                0                1                1                0                0                0                0                1                0   
  2            4       7  1.0006  0.0040   1.2433                7               12                7               12                6               10                7               11               25               25               13               23                 4                 6             3            10             4            11            1            5            5           11         8751            0             0             2             2             0             0             0               1               1               16               21               7               0               1               2                 7                16                0                1                2                3                7                1                0                0                0                0                0                1                1                0                0                0                0                1                0   
  3  

In [40]:
#Separate Train data and test data
trainData = td1.xs(0)
testData = td1.xs(1)
print(trainData.shape)
print(testData.shape)

trainData=pd.concat([trainData,Target_Train_Cols], axis=1)
print(trainData.shape)


(260753, 293)
(173836, 293)
(260753, 294)


In [41]:
export_csv = trainData.to_csv(r'/gdrive/My Drive/CIS508/Assignment-3/Preprocess_Train.csv')
exporttest_csv = testData.to_csv(r'/gdrive/My Drive/CIS508/Assignment-3/Preprocess_Test.csv')